In [41]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate
from langchain import SerpAPIWrapper, LLMChain
from langchain.chat_models import AzureChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, HumanMessage
import re
import openai
import string
import ast
import sqlite3
from datetime import timedelta
import os
import pandas as pd
import numpy as np
import random
from urllib import parse
import re
import json
from sqlalchemy import create_engine  
import sqlalchemy as sql
from plotly.graph_objects import Figure as PlotlyFigure
from matplotlib.figure import Figure as MatplotFigure
import time
from langchain.utilities import PythonREPL
import sys
from io import StringIO


In [115]:
def execute_sql_query(query, limit=10000):  
    db_path = "../data/northwind.db"
    engine = create_engine(f'sqlite:///{db_path}')  
    result = pd.read_sql_query(query, engine)
    result = result.infer_objects()
    for col in result.columns:  
        if 'date' in col.lower():  
            result[col] = pd.to_datetime(result[col], errors="ignore")  

    if limit is not None:  
        result = result.head(limit)  # limit to save memory  

    return result  
def get_table_schema(any=None):
  
  
    # Define the SQL query to retrieve table and column information 
    sql_query = """    
    SELECT m.name AS TABLE_NAME, p.name AS COLUMN_NAME, p.type AS DATA_TYPE  
    FROM sqlite_master AS m  
    JOIN pragma_table_info(m.name) AS p  
    WHERE m.type = 'table'  
    """  
    # Execute the SQL query and store the results in a DataFrame  
    df = execute_sql_query(sql_query)  
    output=[]
    # Initialize variables to store table and column information  
    current_table = ''  
    columns = []  
    
    # Loop through the query results and output the table and column information  
    for index, row in df.iterrows():
        table_name = f"{row['TABLE_NAME']}" 

        column_name = row['COLUMN_NAME']  
        data_type = row['DATA_TYPE']   
        if " " in table_name:
            table_name= f"[{table_name}]" 
        column_name = row['COLUMN_NAME']  
        if " " in column_name:
            column_name= f"[{column_name}]" 

        # If the table name has changed, output the previous table's information  
        if current_table != table_name and current_table != '':  
            output.append(f"table: {current_table}, columns: {', '.join(columns)}")  
            columns = []  
        
        # Add the current column information to the list of columns for the current table  
        columns.append(f"{column_name} {data_type}")  
        
        # Update the current table name  
        current_table = table_name  
    
    # Output the last table's information  
    output.append(f"table: {current_table}, columns: {', '.join(columns)}")
    output = "\n ".join(output)
    return output

def run_python(python_code):

    
    python_code=python_code.strip().strip("```python")
    """Run command and returns anything printed."""
    # sys.stderr.write("EXECUTING PYTHON CODE:\n---\n" + command + "\n---\n")
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()
    try:
        exec(python_code, globals())
        sys.stdout = old_stdout
        output = mystdout.getvalue()
    except Exception as e:
        sys.stdout = old_stdout
        output = "encountered error: "+str(e)
    # sys.stderr.write("PYTHON OUTPUT: \"" + output + "\"\n")
    if len(output)==0:
        output = "python program run successfully without any output. If you want to observe output, you need to use print"
    return output
def persist(df, name):
    df.to_parquet(name)
    # print(f"persist df under {name}")
    return f"persist the input data under {name}"
def load(name):
    return pd.read_parquet(name)

In [117]:
get_table_schema(None)

'table: Categories, columns: CategoryID INTEGER, CategoryName TEXT, Description TEXT, Picture BLOB\n table: sqlite_sequence, columns: name , seq \n table: CustomerCustomerDemo, columns: CustomerID TEXT, CustomerTypeID TEXT\n table: CustomerDemographics, columns: CustomerTypeID TEXT, CustomerDesc TEXT\n table: Customers, columns: CustomerID TEXT, CompanyName TEXT, ContactName TEXT, ContactTitle TEXT, Address TEXT, City TEXT, Region TEXT, PostalCode TEXT, Country TEXT, Phone TEXT, Fax TEXT\n table: Employees, columns: EmployeeID INTEGER, LastName TEXT, FirstName TEXT, Title TEXT, TitleOfCourtesy TEXT, BirthDate DATE, HireDate DATE, Address TEXT, City TEXT, Region TEXT, PostalCode TEXT, Country TEXT, HomePhone TEXT, Extension TEXT, Photo BLOB, Notes TEXT, ReportsTo INTEGER, PhotoPath TEXT\n table: EmployeeTerritories, columns: EmployeeID INTEGER, TerritoryID TEXT\n table: [Order Details], columns: OrderID INTEGER, ProductID INTEGER, UnitPrice NUMERIC, Quantity INTEGER, Discount REAL\n tab

In [118]:
# Import things that are needed generically
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain import LLMMathChain, SerpAPIWrapper

tools = [
    Tool(
        name="Get database schema",
        func=get_table_schema,
        description="list tables and columns information from the database"
    ),

    # Tool(
    #     name="execute sql query",
    #     func=execute_sql_query,
    #     description="given a sql query string, it will execute the query and return the result in pandas dataframe format",
    # ),
    Tool(
        name="run python code",
        func=run_python,
        description="given a python code, it will execute the query and return the result in pandas dataframe format.If you want to see the output of a value, you should print it out with `print(...)`.",
    )

]



In [119]:
template = """Your are a data engineer in a data science team. Your team is given a task by business to work on. You have a team leader and a data scientist colleages. 
You know source systems very well and is tasked to prepare the dataset according to the instruction from the team lead. The data scientist has skills in data science to find insights from the dataset prepared by you.
You have access to the following tools:

{tools}
You  analyze the ask to understand what is required. Once you understand the data that is required, you will retrieve the database schema and write query inside python code 
to retrieve data and perform any additional cleaning and transformation step. Finally, you persist result data for use by your teamm and summarize information about the dataset(s) to your team leader.
You are given following  functions to use in your python code help you retrieve data and persist your result for use by your data scientist.
    1. execute_sql_query(sql_query: str): A Python function can query data from the database given the query. 
        - To use this function that you need to create a sql query which has to be syntactically correct for SQLITE. 
        - execute_sql_query returns a Python pandas dataframe contain the results of the query.
    2. persist(df: Pandas, name:str): this function help you persist pandas dataframe to a storage for later use
    3. load(name): this function is to load a previously persisted dataset. It returns a pandas dataframe
wrap your python code inside ```python ```
Use the following format:

Ask: description of the ask you must perform 
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Ask: {input}
{agent_scratchpad}"""


In [120]:
# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]

In [121]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

In [122]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)
output_parser = CustomOutputParser()

In [123]:
from dotenv import load_dotenv
import openai
import pathlib
from pathlib import Path  # Python 3.6+ only
import os
env_path = Path('.') / 'secrets.env'

load_dotenv(dotenv_path=env_path)


openai.api_type = "azure"
openai.api_version = "2023-03-15-preview" 

chatgpt_deployment = os.environ.get("AZURE_OPENAI_CHATGPT_DEPLOYMENT","gpt-35-turbo")  
gpt4_deployment = os.environ.get("AZURE_OPENAI_GPT4_DEPLOYMENT","gpt-35-turbo")  
endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT","https://resourcenamehere.openai.azure.com/")  
api_key = os.environ.get("AZURE_OPENAI_API_KEY")  
sql_engine = os.environ.get("SQL_ENGINE","sqlite")
dbserver = os.environ.get("SQL_SERVER")
database = os.environ.get("SQL_DATABASE")
db_user = os.environ.get("SQL_USER")
db_password = os.environ.get("SQL_PASSWORD")

llm = AzureChatOpenAI(temperature=0, deployment_name=chatgpt_deployment,openai_api_base=endpoint,openai_api_key=api_key,openai_api_version="2023-03-15-preview" )


In [124]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)


In [125]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [126]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)


In [131]:
ask ="""
Business question: "What are top 10 best selling products in 2016?"
High level description of the required dataset: We need a dataset that includes all sales transactions that occurred in the year 2015. The dataset should include information on the products sold and their corresponding sales quantities and revenues.
Length of history: The dataset should cover the entire year of 2016.
Transformation logics: No transformation logics are required for this dataset.
Please store the dataset using the persist(df: Pandas, name:str) function.
Finally, please provide a summary of the dataset including the number of records, attributes, and summary statistics so that we can prepare the instruction for the data scientist.
Thank you!"""
agent_executor.run(ask)




> Entering new AgentExecutor chain...
Thought: 
To answer the business question, we need to retrieve all sales transactions that occurred in 2016. We also need to join the sales transaction table with the product table to get the product information. We can then group the data by product and sum the sales quantities and revenues to get the total sales for each product. Finally, we can sort the data by total sales and select the top 10 products.

Action: Get database schema
Action Input: None

Observation:table: Categories, columns: CategoryID INTEGER, CategoryName TEXT, Description TEXT, Picture BLOB
 table: sqlite_sequence, columns: name , seq 
 table: CustomerCustomerDemo, columns: CustomerID TEXT, CustomerTypeID TEXT
 table: CustomerDemographics, columns: CustomerTypeID TEXT, CustomerDesc TEXT
 table: Customers, columns: CustomerID TEXT, CompanyName TEXT, ContactName TEXT, ContactTitle TEXT, Address TEXT, City TEXT, Region TEXT, PostalCode TEXT, Country TEXT, Phone TEXT, Fax TEXT


'The top 10 best selling products in 2016 are:\n1. Gorgonzola Telino\n2. Camembert Pierrot\n3. Steeleye Stout\n4. Chartreuse verte\n5. Flotemysost\n6. Raclette Courdavault\n7. Manjimup Dried Apples\n8. Tarte au sucre\n9. Alice Mutton\n10. Carnarvon Tigers'

### Chat Agent

In [283]:
from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [280]:
import os
os.environ["LANGCHAIN_HANDLER"] = "langchain"

In [284]:
tools = [
    Tool(
        name = "Current Search",
        func=lambda message: message,
        description="useful for when you need to answer questions about current events or the current state of the world. the input to this should be a single search term."
    ),
]

In [287]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_chain = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)


In [288]:
agent_chain.run(input="hi, i am bob")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "Hello Bob! How can I assist you today?"
}

> Finished chain.


'Hello Bob! How can I assist you today?'

In [289]:
agent_chain.run(input="what's my name?")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "Your name is Bob."
}

> Finished chain.


'Your name is Bob.'

In [360]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory


from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template="What is a good name for a company that makes {product}?",
            input_variables=["product"],
        )
    )
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chain = LLMChain(llm=llm, prompt=chat_prompt_template,memory=ConversationBufferMemory())
print(chain.run("Hello"))

Hello World Inc.


In [361]:
second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Write a catchphrase for the following company: {company_name}",
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [362]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

# Run the chain specifying only the input variable for the first chain.
catchphrase = overall_chain.run("colorful socks")
print(catchphrase)



> Entering new SimpleSequentialChain chain...
Rainbow Socks Co.
"Step into a world of color with Rainbow Socks Co."

> Finished chain.


"Step into a world of color with Rainbow Socks Co."


In [368]:
from langchain import PromptTemplate, LLMChain
template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [1]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor

In [31]:

# connecting_string = f"Driver={{ODBC Driver 17 for SQL Server}};Server=tcp:{dbserver},1433;Database={database};Uid={db_user};Pwd={db_password}"
# params = parse.quote_plus(connecting_string)

# uri="mssql+pyodbc:///?odbc_connect=%s" % params
uri ="sqlite:///../data/chinook.db"
db = SQLDatabase.from_uri(uri)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)
# from langchain.chains import SQLDatabaseSequentialChain
# chain = SQLDatabaseSequentialChain.from_llm(llm, db, verbose=True)
# agent_executor.run("Show me revenue by month, by customer")
agent_executor.run("show me revenue by month for top 10 customers")




> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: sqlite_stat1, genres, tracks, invoices, invoice_items, albums, customers, media_types, sqlite_sequence, playlists, artists, employees, playlist_track
Thought:I should query the invoices and customers tables to get the revenue by month for the top 10 customers.
Action: query_checker_sql_db
Action Input: "SELECT strftime('%Y-%m', invoices.InvoiceDate) as Month, customers.FirstName || ' ' || customers.LastName as Customer, SUM(invoice_items.UnitPrice * invoice_items.Quantity) as Revenue FROM invoices JOIN customers ON invoices.CustomerId = customers.CustomerId GROUP BY Month, Customer ORDER BY Revenue DESC LIMIT 10;"
Observation: The original query appears to be correct and does not contain any of the common mistakes listed. Therefore, the original query is:

SELECT strftime('%Y-%m', invoices.InvoiceDate) as Month, customers.FirstName || ' ' || customers.LastName as Customer, SUM(invoice_items

'The revenue by month for the top 10 customers is [(\'2013-11\', \'Helena Holý\', 25.859999999999992), (\'2012-08\', \'Richard Cunningham\', 23.859999999999992), (\'2010-02\', \'Ladislav Kovács\', 21.859999999999996), (\'2011-04\', "Hugh O\'Reilly", 21.859999999999992), (\'2011-05\', \'Victor Stevens\', 18.86), (\'2010-01\', \'Astrid Gruber\', 18.859999999999996), (\'2010-01\', \'Luis Rojas\', 17.91), (\'2012-09\', \'František Wichterlová\', 16.86), (\'2012-10\', \'Isabelle Mercier\', 16.86), (\'2010-03\', \'Frank Ralston\', 15.860000000000001)].'

In [24]:
db.dialect

'mssql'